In [2]:
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Carregando a base de dados.

In [4]:
df_edu = pd.read_csv(r"C:\Users\marcf\OneDrive\Documentos\Ciencia de dados\Árvores de decisão e Florestas Aleatórias\random-forest-materiais-apoio\xAPI-Edu-Data.csv")

In [6]:
df_edu.head(10)

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M
5,F,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,42,30,13,70,Yes,Bad,Above-7,M
6,M,KW,KuwaIT,MiddleSchool,G-07,A,Math,F,Father,35,12,0,17,No,Bad,Above-7,L
7,M,KW,KuwaIT,MiddleSchool,G-07,A,Math,F,Father,50,10,15,22,Yes,Good,Under-7,M
8,F,KW,KuwaIT,MiddleSchool,G-07,A,Math,F,Father,12,21,16,50,Yes,Good,Under-7,M
9,F,KW,KuwaIT,MiddleSchool,G-07,B,IT,F,Father,70,80,25,70,Yes,Good,Under-7,M


In [10]:
df_edu.describe()

,raisedhands,VisITedResources,AnnouncementsView,Discussion
count,480.000000,480.000000,480.000000,480.000000
mean,46.775000,54.797917,37.918750,43.283333
std,30.779223,33.080007,26.611244,27.637735
min,0.000000,0.000000,0.000000,1.000000
25%,15.750000,20.000000,14.000000,20.000000
50%,50.000000,65.000000,33.000000,39.000000
75%,75.000000,84.000000,58.000000,70.000000
max,100.000000,99.000000,98.000000,99.000000


Verificando as distribuições de classes

In [11]:
df_edu['Class'].value_counts()

M    211
H    142
L    127
Name: Class, dtype: int64

Verificando os registros nulos

In [12]:
df_edu.isnull().sum()

gender                      0
NationalITy                 0
PlaceofBirth                0
StageID                     0
GradeID                     0
SectionID                   0
Topic                       0
Semester                    0
Relation                    0
raisedhands                 0
VisITedResources            0
AnnouncementsView           0
Discussion                  0
ParentAnsweringSurvey       0
ParentschoolSatisfaction    0
StudentAbsenceDays          0
Class                       0
dtype: int64

Codificando os atributos numéricos

In [13]:
Features = df_edu
Cat_Columns = Features.dtypes.pipe(lambda Features: Features[Features=='object']).index
for col in Cat_Columns:
    label = LabelEncoder()
    Features[col] = label.fit_transform(Features[col])

In [14]:
Features.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,1,4,4,2,1,0,7,0,0,15,16,2,20,1,1,1,2
1,1,4,4,2,1,0,7,0,0,20,20,3,25,1,1,1,2
2,1,4,4,2,1,0,7,0,0,10,7,0,30,0,0,0,1
3,1,4,4,2,1,0,7,0,0,30,25,5,35,0,0,0,1
4,1,4,4,2,1,0,7,0,0,40,50,12,50,0,0,0,2


In [15]:
Features.describe()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,0.635417,4.345833,4.397917,1.345833,2.906250,0.472917,5.256250,0.489583,0.410417,46.775000,54.797917,37.918750,43.283333,0.562500,0.608333,0.602083,1.143750
std,0.481815,2.469265,2.628334,0.603732,2.464267,0.612411,3.388388,0.500413,0.492423,30.779223,33.080007,26.611244,27.637735,0.496596,0.488632,0.489979,0.846312
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.000000,3.000000,1.000000,0.000000,0.000000,3.000000,0.000000,0.000000,15.750000,20.000000,14.000000,20.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,4.000000,4.000000,1.000000,4.000000,0.000000,5.000000,0.000000,0.000000,50.000000,65.000000,33.000000,39.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,4.000000,4.000000,2.000000,5.000000,1.000000,7.000000,1.000000,1.000000,75.000000,84.000000,58.000000,70.000000,1.000000,1.000000,1.000000,2.000000
max,1.000000,13.000000,13.000000,2.000000,9.000000,2.000000,11.000000,1.000000,1.000000,100.000000,99.000000,98.000000,99.000000,1.000000,1.000000,1.000000,2.000000


Separando os dados e classes

In [17]:
dataset = df_edu.drop('Class', axis=1)

In [18]:
classes = df_edu['Class']

# Random Forest vs Árvore de Decisão

Resultados Random Forest

In [19]:
random_clf = RandomForestClassifier(random_state=1,n_estimators=100)

In [29]:
resultados_random = cross_val_predict(random_clf, dataset,classes,cv=5)

In [30]:
print(classification_report(classes,resultados_random))

              precision    recall  f1-score   support

           0       0.65      0.64      0.65       142
           1       0.77      0.78      0.77       127
           2       0.63      0.63      0.63       211

    accuracy                           0.67       480
   macro avg       0.68      0.68      0.68       480
weighted avg       0.67      0.67      0.67       480



Resultados Decision Tree

In [22]:
tree_clf = DecisionTreeClassifier(random_state=1)

In [23]:
tree_clf

DecisionTreeClassifier(random_state=1)

In [27]:
resultados_tree = cross_val_predict(tree_clf, dataset,classes,cv=5)

In [28]:
print(classification_report(classes,resultados_tree))

              precision    recall  f1-score   support

           0       0.50      0.61      0.55       142
           1       0.74      0.68      0.70       127
           2       0.54      0.49      0.52       211

    accuracy                           0.57       480
   macro avg       0.59      0.59      0.59       480
weighted avg       0.58      0.57      0.58       480



Verificando Overfitting

In [31]:
x_train, x_test, y_train, y_test = train_test_split(df_edu.drop('Class', axis=1),
                                                    df_edu['Class'],
                                                    test_size=0.3,
                                                    random_state = 1)

In [72]:
def compara_modelos_random_forest(maxdepth):
    if maxdepth == 0:
        rf = RandomForestClassifier(n_estimators = 100, random_state=1)
    else:
        rf = RandomForestClassifier(n_estimators = 100, random_state=1, max_depth=maxdepth)
    rf.fit(x_train, y_train)
    train_score = rf.score(x_train, y_train)
    test_score = rf.score(x_test, y_test)
    return train_score, test_score

In [73]:
print('{:10} {:20} {:20}'.format('depth', 'Training Score', 'Testing Score'))
print('{:10} {:20} {:20}'.format('-----', '--------------', '-------------'))
depth = range(1,16)

for profundidade in depth:
    if profundidade < 10:
        print('{:1}         {}'.format(profundidade,str(compara_modelos_random_forest(profundidade))))
    else:
        print('{:1}        {}'.format(profundidade,str(compara_modelos_random_forest(profundidade))))
        
print('{:1}      {}'.format('Full',str(compara_modelos_random_forest(0))))

depth      Training Score       Testing Score       
-----      --------------       -------------       
1         (0.6666666666666666, 0.5486111111111112)
2         (0.75, 0.6180555555555556)
3         (0.8244047619047619, 0.6805555555555556)
4         (0.8720238095238095, 0.7152777777777778)
5         (0.8958333333333334, 0.75)
6         (0.9345238095238095, 0.7847222222222222)
7         (0.9702380952380952, 0.7916666666666666)
8         (0.9821428571428571, 0.7638888888888888)
9         (1.0, 0.7847222222222222)
10        (1.0, 0.7569444444444444)
11        (1.0, 0.7916666666666666)
12        (1.0, 0.7986111111111112)
13        (1.0, 0.7847222222222222)
14        (1.0, 0.7986111111111112)
15        (1.0, 0.7986111111111112)
Full      (1.0, 0.7986111111111112)


In [74]:
def compara_modelos_random_forest_discrepancy(maxdepth):
    if maxdepth == 0:
        rf = RandomForestClassifier(n_estimators = 100, random_state=1)
    else:
        rf = RandomForestClassifier(n_estimators = 100, random_state=1, max_depth=maxdepth)
    rf.fit(x_train, y_train)
    train_score = rf.score(x_train, y_train)
    test_score = rf.score(x_test, y_test)
    return train_score - test_score

In [75]:
print('{:10} {:20} {:20} {:20}'.format('depth', 'Training Score', 'Testing Score', 'Discrepancy Training x Testing'))
print('{:10} {:20} {:20} {:20}'.format('-----', '--------------', '-------------', '------------------------------'))
depth = range(1,16)

for profundidade in depth:
    if profundidade < 10:
        print('{:1}         {}   {}'.format(profundidade,str(compara_modelos_random_forest(profundidade)),
                                       compara_modelos_random_forest_discrepancy(profundidade)
                                      )
             )
    else:
        print('{:1}        {}    {}'.format(profundidade,str(compara_modelos_random_forest(profundidade)),
                                      compara_modelos_random_forest_discrepancy(profundidade)
                                     )
             )
        
print('{:1}      {}    {}'.format('Full',str(compara_modelos_random_forest(0)),
                            compara_modelos_random_forest_discrepancy(0)
                           )
     )

depth      Training Score       Testing Score        Discrepancy Training x Testing
-----      --------------       -------------        ------------------------------
1         (0.6666666666666666, 0.5486111111111112)   0.11805555555555547
2         (0.75, 0.6180555555555556)   0.13194444444444442
3         (0.8244047619047619, 0.6805555555555556)   0.14384920634920628
4         (0.8720238095238095, 0.7152777777777778)   0.15674603174603174
5         (0.8958333333333334, 0.75)   0.14583333333333337
6         (0.9345238095238095, 0.7847222222222222)   0.14980158730158732
7         (0.9702380952380952, 0.7916666666666666)   0.1785714285714286
8         (0.9821428571428571, 0.7638888888888888)   0.21825396825396826
9         (1.0, 0.7847222222222222)   0.2152777777777778
10        (1.0, 0.7569444444444444)    0.24305555555555558
11        (1.0, 0.7916666666666666)    0.20833333333333337
12        (1.0, 0.7986111111111112)    0.20138888888888884
13        (1.0, 0.7847222222222222)    0.21

In [76]:
def compara_modelos_decision_tree(maxdepth):
    if maxdepth == 0:
        df = DecisionTreeClassifier(random_state=1)
    else:
        df = DecisionTreeClassifier(random_state=1, max_depth=maxdepth)
    df.fit(x_train, y_train)
    train_score = df.score(x_train, y_train)
    test_score = df.score(x_test, y_test)
    return train_score, test_score

In [77]:
print('{:10} {:20} {:20}'.format('depth', 'Training Score', 'Testing Score'))
print('{:10} {:20} {:20}'.format('-----', '--------------', '-------------'))
depth = range(1,16)

for profundidade in depth:
    if profundidade < 10:
        print('{:1}         {}'.format(profundidade,str(compara_modelos_decision_tree(profundidade))))
    else:
        print('{:1}        {}'.format(profundidade,str(compara_modelos_decision_tree(profundidade))))
        
print('{:1}      {}'.format('Full',str(compara_modelos_decision_tree(0))))

depth      Training Score       Testing Score       
-----      --------------       -------------       
1         (0.5892857142857143, 0.5)
2         (0.6398809523809523, 0.6805555555555556)
3         (0.7321428571428571, 0.7013888888888888)
4         (0.7916666666666666, 0.7430555555555556)
5         (0.8690476190476191, 0.6805555555555556)
6         (0.8988095238095238, 0.7152777777777778)
7         (0.9404761904761905, 0.7152777777777778)
8         (0.9702380952380952, 0.7083333333333334)
9         (0.9821428571428571, 0.6875)
10        (0.9910714285714286, 0.6875)
11        (0.9940476190476191, 0.6944444444444444)
12        (1.0, 0.6944444444444444)
13        (1.0, 0.6944444444444444)
14        (1.0, 0.6944444444444444)
15        (1.0, 0.6944444444444444)
Full      (1.0, 0.6944444444444444)


In [80]:
def compara_modelos_decision_tree_discrepancy(maxdepth):
    if maxdepth == 0:
        df = DecisionTreeClassifier(random_state=1)
    else:
        df = DecisionTreeClassifier(random_state=1, max_depth=maxdepth)
    df.fit(x_train, y_train)
    train_score = df.score(x_train, y_train)
    test_score = df.score(x_test, y_test)
    return train_score - test_score

In [81]:
print('{:10} {:20} {:20} {:20}'.format('depth', 'Training Score', 'Testing Score', 'Discrepancy Training x Testing'))
print('{:10} {:20} {:20} {:20}'.format('-----', '--------------', '-------------', '------------------------------'))
depth = range(1,16)

for profundidade in depth:
    if profundidade < 10:
        print('{:1}         {}   {}'.format(profundidade,str(compara_modelos_decision_tree(profundidade)),
                                            compara_modelos_decision_tree_discrepancy(profundidade)
                                      )
             )
    else:
        print('{:1}        {}    {}'.format(profundidade,str(compara_modelos_decision_tree(profundidade)),
                                            compara_modelos_decision_tree_discrepancy(profundidade)
                                     )
             )
        
print('{:1}      {}    {}'.format('Full',str(compara_modelos_decision_tree(0)),
                                  compara_modelos_decision_tree_discrepancy(0)
                           )
     )

depth      Training Score       Testing Score        Discrepancy Training x Testing
-----      --------------       -------------        ------------------------------
1         (0.5892857142857143, 0.5)   0.0892857142857143
2         (0.6398809523809523, 0.6805555555555556)   -0.04067460317460325
3         (0.7321428571428571, 0.7013888888888888)   0.030753968253968256
4         (0.7916666666666666, 0.7430555555555556)   0.04861111111111105
5         (0.8690476190476191, 0.6805555555555556)   0.1884920634920635
6         (0.8988095238095238, 0.7152777777777778)   0.18353174603174605
7         (0.9404761904761905, 0.7152777777777778)   0.22519841269841268
8         (0.9702380952380952, 0.7083333333333334)   0.26190476190476186
9         (0.9821428571428571, 0.6875)   0.2946428571428571
10        (0.9910714285714286, 0.6875)    0.3035714285714286
11        (0.9940476190476191, 0.6944444444444444)    0.29960317460317465
12        (1.0, 0.6944444444444444)    0.3055555555555556
13        

In [93]:
def compara_modelos_random_forest_classifier(maxdepth):
    if maxdepth == 0:
        rf = RandomForestClassifier(n_estimators = 100, random_state=1)
    else:
        rf = RandomForestClassifier(n_estimators = 100, random_state=1, max_depth=maxdepth)
    rf.fit(x_train, y_train)
    train_score = rf.score(x_train, y_train)
    test_score = rf.score(x_test, y_test)
    if (train_score - test_score) < 0.10:
        classificador = 'Não sofre overfitting'
    else:
        classificador = 'Sofre overfitting'
    return classificador

In [94]:
print('{:10} {:20} {:20} {:20} {:20}'.format('depth', 'Training Score', 'Testing Score', 'Discrepancy Training x Testing', 'Overfitting'))
print('{:10} {:20} {:20} {:20} {:20}'.format('-----', '--------------', '-------------', '------------------------------', '-----------'))
depth = range(1,16)

for profundidade in depth:
    if profundidade < 10:
        print('{:1}         {}   {}             {}'.format(profundidade,
                                                           str(compara_modelos_random_forest(profundidade)),
                                                           compara_modelos_random_forest_discrepancy(profundidade),
                                                           compara_modelos_random_forest_classifier(profundidade)
                                      )
             )
    else:
        print('{:1}        {}    {}             {}'.format(profundidade,
                                                           str(compara_modelos_random_forest(profundidade)),
                                                           compara_modelos_random_forest_discrepancy(profundidade),
                                                           compara_modelos_random_forest_classifier(profundidade)
                                     )
             )
        
print('{:1}      {}    {}           {}'.format('Full',
                                               str(compara_modelos_random_forest(0)),
                                               compara_modelos_random_forest_discrepancy(0),
                                               compara_modelos_random_forest_classifier(0)
                           )
     )

depth      Training Score       Testing Score        Discrepancy Training x Testing Overfitting         
-----      --------------       -------------        ------------------------------ -----------         
1         (0.6666666666666666, 0.5486111111111112)   0.11805555555555547             Sofre overfitting
2         (0.75, 0.6180555555555556)   0.13194444444444442             Sofre overfitting
3         (0.8244047619047619, 0.6805555555555556)   0.14384920634920628             Sofre overfitting
4         (0.8720238095238095, 0.7152777777777778)   0.15674603174603174             Sofre overfitting
5         (0.8958333333333334, 0.75)   0.14583333333333337             Sofre overfitting
6         (0.9345238095238095, 0.7847222222222222)   0.14980158730158732             Sofre overfitting
7         (0.9702380952380952, 0.7916666666666666)   0.1785714285714286             Sofre overfitting
8         (0.9821428571428571, 0.7638888888888888)   0.21825396825396826             Sofre overfit

In [95]:
def compara_modelos_decision_tree_classifier(maxdepth):
    if maxdepth == 0:
        df = DecisionTreeClassifier(random_state=1)
    else:
        df = DecisionTreeClassifier(random_state=1, max_depth=maxdepth)
    df.fit(x_train, y_train)
    train_score = df.score(x_train, y_train)
    test_score = df.score(x_test, y_test)
    if (train_score - test_score) < 0.10:
        classificador = 'Não sofre overfitting'
    else:
        classificador = 'Sofre overfitting'
    return classificador

In [96]:
print('{:10} {:20} {:20} {:20} {:20}'.format('depth', 'Training Score', 'Testing Score', 'Discrepancy Training x Testing', 'Overfitting'))
print('{:10} {:20} {:20} {:20} {:20}'.format('-----', '--------------', '-------------', '------------------------------', '-----------'))
depth = range(1,16)

for profundidade in depth:
    if profundidade < 10:
        print('{:1}         {}   {}             {}'.format(profundidade,
                                                           str(compara_modelos_decision_tree(profundidade)),
                                                           compara_modelos_decision_tree_discrepancy(profundidade),
                                                           compara_modelos_decision_tree_classifier(profundidade)
                                      )
             )
    else:
        print('{:1}        {}    {}             {}'.format(profundidade,
                                                           str(compara_modelos_decision_tree(profundidade)),
                                                           compara_modelos_decision_tree_discrepancy(profundidade),
                                                           compara_modelos_decision_tree_classifier(profundidade)
                                     )
             )
        
print('{:1}      {}    {}           {}'.format('Full',
                                               str(compara_modelos_decision_tree(0)),
                                               compara_modelos_decision_tree_discrepancy(0),
                                               compara_modelos_decision_tree_classifier(0)
                           )
     )

depth      Training Score       Testing Score        Discrepancy Training x Testing Overfitting         
-----      --------------       -------------        ------------------------------ -----------         
1         (0.5892857142857143, 0.5)   0.0892857142857143             Não sofre overfitting
2         (0.6398809523809523, 0.6805555555555556)   -0.04067460317460325             Não sofre overfitting
3         (0.7321428571428571, 0.7013888888888888)   0.030753968253968256             Não sofre overfitting
4         (0.7916666666666666, 0.7430555555555556)   0.04861111111111105             Não sofre overfitting
5         (0.8690476190476191, 0.6805555555555556)   0.1884920634920635             Sofre overfitting
6         (0.8988095238095238, 0.7152777777777778)   0.18353174603174605             Sofre overfitting
7         (0.9404761904761905, 0.7152777777777778)   0.22519841269841268             Sofre overfitting
8         (0.9702380952380952, 0.7083333333333334)   0.2619047619047

## Tunning do Modelo para garantir o melhor Desempenho

Como encontrar os melhores valores para os parâmetros do modelo?

RandomForestClassifier(n_estimators=?, criterion='gini' ou 'entropy', max_depth=?, min_samples_split=?, min_samples_leaf=?)

In [97]:
from sklearn.model_selection import GridSearchCV

Lista de possíveis valores de estimators ou quantidade de árvores da floresta

In [98]:
valores_estimators = [10, 20, 50, 100, 150]

Lista de possíveis valores para o critério de divisão

In [100]:
valores_criterion = ['gini', 'entropy']

Lista de possíveis valores para a profundidade máxima de cada árvore

In [101]:
valores_max_depth = [10, 20, 50, 100]

Lista de possíveis valores para os parametros min_samples_split e min_samples_leaf

In [103]:
valores_min_samples_split = [2, 5, 10, 15]
valores_min_samples_leaf = [1, 5, 10, 15]

Define um dicionário que recebe as listas de parâmetros e valores.

In [104]:
parametros_grid = dict(n_estimators=valores_estimators,
                       criterion=valores_criterion,
                       max_depth=valores_max_depth,
                       min_samples_split=valores_min_samples_split,
                       min_samples_leaf=valores_min_samples_leaf
                       )

Dicionário com os parametros que serão utilizados no grid.

In [105]:
parametros_grid

{'n_estimators': [10, 20, 50, 100, 150],
 'criterion': ['gini', 'entropy'],
 'max_depth': [10, 20, 50, 100],
 'min_samples_split': [2, 5, 10, 15],
 'min_samples_leaf': [1, 5, 10, 15]}

Instancia o GridSearch com o modelo a ser utilizado, parametros, número de folds e scoring

In [106]:
rf = RandomForestClassifier()

In [107]:
grid = GridSearchCV(rf, parametros_grid, cv=5, scoring='accuracy')

Aplica o GridSearch passando as features e classes

In [111]:
grid.fit(df_edu.drop('Class', axis=1),df_edu['Class'])

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 50, 100],
                         'min_samples_leaf': [1, 5, 10, 15],
                         'min_samples_split': [2, 5, 10, 15],
                         'n_estimators': [10, 20, 50, 100, 150]},
             scoring='accuracy')

Imprime os scores por combinações _(o atributo grid_scores_foi depreciado e substituído por cv_results__)

In [115]:
grid.cv_results_

{'mean_fit_time': array([0.02594976, 0.03519044, 0.07774415, 0.15448532, 0.22752037,
        0.01696815, 0.03210135, 0.08163257, 0.16833849, 0.22944155,
        0.01672754, 0.02992473, 0.07346692, 0.13915439, 0.21389318,
        0.01628308, 0.03040805, 0.06965737, 0.13526931, 0.20361829,
        0.01583443, 0.03059964, 0.06999364, 0.13458176, 0.20396929,
        0.02165179, 0.0315403 , 0.07098436, 0.15144958, 0.22560511,
        0.0227777 , 0.04522324, 0.08189516, 0.15424843, 0.21913538,
        0.01680408, 0.03245506, 0.07557912, 0.15110092, 0.25455666,
        0.01748114, 0.03151226, 0.08334389, 0.12912726, 0.22732415,
        0.02178774, 0.03071399, 0.06607909, 0.1358593 , 0.22663598,
        0.01528277, 0.02795982, 0.06477008, 0.12911482, 0.19309125,
        0.01473837, 0.02724433, 0.06423035, 0.14070177, 0.24873271,
        0.0150064 , 0.10936756, 0.11629677, 0.19410939, 0.27461324,
        0.02510834, 0.05529203, 0.06918201, 0.15380554, 0.22465487,
        0.01796622, 0.0342598 ,

Verificando os melhores parâmetros

In [116]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 20,
 'min_samples_leaf': 15,
 'min_samples_split': 5,
 'n_estimators': 20}

Verificando o melhor Score,

In [117]:
grid.best_score_

0.725